<h1 align="center"> Crop image </hi>

Crop a raster georeferenced image so that it can be divided in square patches of a given size. The ouput is also georeferenced.

## Imports

In [1]:
import numpy as np
import os
import rasterio
from rasterio.transform import from_gcps
from rasterio.control import GroundControlPoint as GCP

## Set paths

In [2]:
img_folder = '/lithology_paper/S1/Processed'
save_folder = '/lithology_paper/S1/Cropped'

## Utilities

In [3]:
def pixcoords_subset(img, patch_size):
    '''
    Get the pixel coordinates of subset of an image, so that the subset is divisible with a given patch size.
    Inputs: img(rasterio dataset object) - the raster image
            patch_size(int) - the number that should divide the image perfectly
    Ouput: pix_coords(list) - the pixel coordinates of the subset area  in the following order:
                              [left, right, upper, lower].
    '''
    #Calculate subset area coordinates
    rows, cols = img.height, img.width
    extra_rows = rows % patch_size
    extra_cols = cols % patch_size
    new_rows = rows - extra_rows
    new_cols = cols - extra_cols
    
    #Assign coordinates
    left = 0
    right = new_cols
    upper = 0
    lower = new_rows
    pix_coords = [left, right, upper, lower]
    return pix_coords


def calc_transform_subs(img, pix_coords):
    '''
    Calculate the affine transformation that describes the relationship between the pixel coordinate system 
    and the geospatial reference system that describe a SUBSET of a given georeferenced image.
    
    Inputs: pix_coords(list) - contains the bounding box coordinates of the subset area in the pixel 
                               corrdinate system of the original image with the following order:
                               [left, right, upper, lower]
                               
    Output: transform(affine object)   - the affine transform which is a rasterio Affine object.
    '''
    #Unpack
    left = pix_coords[0]
    right = pix_coords[1]
    upper = pix_coords[2]
    lower = pix_coords[3]
    
    #Width and height of the subset area in pixels
    width = right - left
    height = lower - upper
    
    #Get the xy bound box coordinates in the geospatial reference system
    ul = img.xy(left, upper)
    ur = img.xy(right, upper)
    ll = img.xy(left, lower)
    lr = img.xy(right, lower)
    
    #Create the corresponding ground control points
    gcps = [GCP(0, 0, *ul),
            GCP(width,0 , *ur),
            GCP(0, height, *ll),
            GCP(width, height, *lr)]
    
    #Calculate transform
    transform = from_gcps(gcps)
    
    return transform

def write_raster(img_np, save_path, crs, transform, driver = 'GTiff'):
    '''
    Write a georeferenced raster to a geotiff file.
    Inputs: img_np(numpy array) - raster image to write
            save_path(string) - full path to save file
            crs(crs rasterio object) - coordinate reference system
            transform(affine rasterio object) - affine transformation between pixel and ground coordinates
            driver(string)(default = GTiff) - the name of the gdal driver
    '''
    with rasterio.open(save_path,
                       'w',
                       driver = driver,
                       height = img_np.shape[1],
                       width = img_np.shape[2],
                       count = img_np.shape[0],
                       dtype = img_np.dtype,
                       crs = crs,
                       transform = transform
    ) as dst:
        dst.write(img_np)

def crop(img, patch_size):
    '''
    Crop an image based so that it is divided perfectly by a number.
    Inputs: img(rasterio dataset object) - the image to crop
            patch_size(int) - the number that should perfectly divide the image
    Output: img_np(numpy array) - cropped image
    '''
    rows, cols = img.height, img.width
    extra_rows = rows % patch_size
    extra_cols = cols % patch_size
    new_rows = rows - extra_rows
    new_cols = cols - extra_cols

    window = rasterio.windows.Window(0, 0, new_cols, new_rows)
    
    bands_ls = []
    for i in range (img.count):
        band = img.read(i+1, window = window)
        bands_ls.append(band)
        img_np = np.stack(bands_ls)
        
    return img_np

## Main

In [ ]:
i = 1
print('Cropping...')
for img_name in os.listdir(img_folder):
    total = len(os.listdir(img_folder))
    print('{}/{}:{}'.format(i, total, img_name))
    img_path = os.path.join(img_folder, img_name)
    img = rasterio.open(img_path)  
    
    cropped_img = crop(img, 128)
    subset_coords = pixcoords_subset(img, 128)
    transform = calc_transform_subs(img, subset_coords)
    save_path = os.path.join(save_folder, img_name)
    crs = img.crs

    write_raster(cropped_img, save_path, crs, transform)
    img.close()
    i = i + 1

print()
print('Done!')